<a href="https://colab.research.google.com/github/rediahmds/eco-sort/blob/main/train/build_dataset_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to Google Drive

In [43]:
USE_COLAB = False

if USE_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
else:
    from pydrive2.auth import GoogleAuth
    from pydrive2.drive import GoogleDrive

    try:
        !pip install PyDrive2

        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        secret_value_0 = user_secrets.get_secret("GCP_CLIENT_SECRET")

        with open("client_secrets.json", "w") as f:
            f.write(secret_value_0)


        gauth = GoogleAuth()
        gauth.CommandLineAuth()

        drive = GoogleDrive(gauth)
    except FileNotFoundError as fne:
        print("Could not find credentials from GCP named client_secrets.json")
        print("Error:", fne)


Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?client_id=991309652978-hpmqspbdt0bccglf6iklc3er1d9naa8q.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=online&response_type=code



Enter verification code:  4/1AVMBsJg2mS_H411Rn12AFHu2NuotDzPiVw2HeYoOc9uNfxh_Dw6Vyr2ZInc


Authentication successful.


## Prepare Dataset

### Download Dataset from Kaggle

The dataset comes from different accounts on Kaggle.

In [1]:
!pip install kagglehub

In [29]:
import kagglehub

# alistair_ds = kagglehub.dataset_download("alistairking/recyclable-and-household-waste-classification")
# print("Path to dataset files:", alistair_ds)

# mostafa_ds = kagglehub.dataset_download("mostafaabla/garbage-classification")
# print("Path to dataset files:", mostafa_ds)

# joe_ds = kagglehub.dataset_download("joebeachcapital/realwaste")
# print("Path to dataset files:", joe_ds)

# glhdamar_ds = kagglehub.dataset_download("glhdamar/new-trash-classfication-dataset")
# print("Path to dataset files:", glhdamar_ds)

# arkadiy_ds = kagglehub.dataset_download("arkadiyhacks/drinking-waste-classification")

# Download latest version
techash_ds = kagglehub.dataset_download("techsash/waste-classification-data")
print("Path to dataset files:", techash_ds)

Path to dataset files: /kaggle/input/waste-classification-data


In [3]:
!git clone https://github.com/nikhilvenkatkumsetty/TrashBox.git

Cloning into 'TrashBox'...
remote: Enumerating objects: 17962, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 17962 (delta 12), reused 0 (delta 0), pack-reused 17941 (from 1)
Receiving objects: 100% (17962/17962), 2.66 GiB | 54.43 MiB/s, done.
Resolving deltas: 100% (161/161), done.
Updating files: 100% (28564/28564), done.


### Show Directory Tree

In [4]:
from pathlib import Path

def print_directory_tree(root: Path, prefix: str = ""):
    """
    Mencetak struktur direktori dengan tampilan seperti pohon.
    Hanya menampilkan folder (tanpa file).
    """
    subdirs = sorted([p for p in root.iterdir() if p.is_dir()])
    for i, subdir in enumerate(subdirs):
        connector = "└── " if i == len(subdirs) - 1 else "├── "
        print(f"{prefix}{connector}{subdir.name}")
        extension = "    " if i == len(subdirs) - 1 else "│   "
        print_directory_tree(subdir, prefix + extension)

In [32]:
# Path ke folder utama
# alistair_path = Path(alistair_ds) / "images" / "images"
# mostafa_path = Path(mostafa_ds) / "garbage_classification"
# joe_path = Path(joe_ds) / "realwaste-main" / "RealWaste"
# glhdamar_path = Path(glhdamar_ds) / "new-dataset-trash-type-v2"
techash_path = Path(techash_ds) / "TRAIN" 

# print(alistair_path.name)
# print_directory_tree(alistair_path)

# print(mostafa_path.name)
# print_directory_tree(mostafa_path)

# print(joe_path.name)
# print_directory_tree(joe_path)

# print(glhdamar_path.name)
# print_directory_tree(glhdamar_path)
print(techash_path.name)
print_directory_tree(techash_path.name)


O


AttributeError: 'str' object has no attribute 'iterdir'

### Download Existing Dataset from Google Drive

In [6]:
if USE_COLAB:
    !cp /content/drive/MyDrive/Cool Lee Yeah/8th Semester/Skripsi/Datasets/dataset.7z .
else:
    from kaggle_secrets import UserSecretsClient
    GDRIVE_DATASET_FILE_ID = user_secrets.get_secret("GDRIVE_DATASET_FILE_ID")
    file_id = GDRIVE_DATASET_FILE_ID
    local_filename = "dataset.7z" # save as
    try:
        file_to_download = drive.CreateFile({'id': file_id})

        print(f"⬇️ Downloading file: '{file_to_download['title']}'...")
        file_to_download.GetContentFile(local_filename)

        print(f"✅ Downloaded successfully and saved as '{local_filename}'")

    except Exception as e:
        print(f"Error: {e}")


⬇️ Downloading file: 'dataset_no-styro.7z'...
✅ Downloaded successfully and saved as 'dataset.7z'


In [17]:
!7z x dataset.7z


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,4 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Sca        1 file, 1234923334 bytes (1178 MiB)

Extracting archive: dataset.7z
--
Path = dataset.7z
Type = 7z
Physical Size = 1234923334
Headers Size = 178477
Method = LZMA2:24
Solid = +
Blocks = 1

      0%      0% 582 - dataset/train/background/158.jp                                            0% 817 - dataset/train/background/37.j                                          0% 1211 - dataset/train/background/724.j                                            0% 1338 - dataset/train/background/839.j                                            1% 1531 - dataset/train/glass/Glass_112.jp                                              1% 1608 - dataset/train/glass/Glass_182.jp                                              2% 1650 - dataset/train/glass/Glass_22.j   

In [8]:
print_directory_tree(Path("dataset/train"))

├── background
├── glass
├── metal
├── organic
├── paper
├── plastic
└── textiles


### Merge All Dataset

Copy from different sources.

In [9]:
from pathlib import Path
import shutil
import random

def copy_n_files(src_dir, dst_dir, n, randomize=False):
    src_path = Path(src_dir)
    dst_path = Path(dst_dir)

    # Buat folder tujuan jika belum ada
    dst_path.mkdir(parents=True, exist_ok=True)

    # Ambil semua file dari direktori sumber
    all_files = [f for f in src_path.iterdir() if f.is_file()]

    # Pastikan n tidak lebih besar dari jumlah file
    n = min(n, len(all_files))

    # Tentukan file mana yang akan disalin
    if randomize:
        files_to_copy = random.sample(all_files, n)
    else:
        files_to_copy = sorted(all_files)[:n]

    # Copy file satu per satu
    for file in files_to_copy:
        shutil.copy(file, dst_path)
        print(f"Copied: {file.name}")

    print(f"\nTotal {n} files copied from '{src_dir}' to '{dst_dir}' (random: {randomize}).")

In [38]:
# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Plastic", "dataset/train/plastic", 920, randomize=True)

# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Paper", "dataset/train/paper", 500, randomize=True)
# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Cardboard", "dataset/train/paper", 461, randomize=True)

# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Food Organics", "dataset/train/organic", 411, randomize=True)
# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Vegetation", "dataset/train/organic", 436, randomize=True)

# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Glass", "dataset/train/glass", 420, randomize=True)

# copy_n_files(f"{joe_ds}/realwaste-main/RealWaste/Metal", "dataset/train/metal", 420, randomize=True)

# copy_n_files(f"/kaggle/working/TrashBox/TrashBox_train_set/metal", "dataset/train/metal", 1800, randomize=True)
# copy_n_files(f"/kaggle/working/TrashBox/TrashBox_train_set/plastic", "dataset/train/plastic", 800, randomize=True)
# copy_n_files(f"/kaggle/working/TrashBox/TrashBox_train_set/glass", "dataset/train/glass", 1000, randomize=True)

copy_n_files(f"/kaggle/input/waste-classification-data/DATASET/TRAIN/O", "dataset/train/organic", 1000, randomize=True)



Copied: O_2799.jpg
Copied: O_6244.jpg
Copied: O_7023.jpg
Copied: O_3182.jpg
Copied: O_11599.jpg
Copied: O_12283.jpg
Copied: O_7547.jpg
Copied: O_12147.jpg
Copied: O_9314.jpg
Copied: O_12450.jpg
Copied: O_2638.jpg
Copied: O_8773.jpg
Copied: O_8247.jpg
Copied: O_9644.jpg
Copied: O_4077.jpg
Copied: O_1113.jpg
Copied: O_4758.jpg
Copied: O_5380.jpg
Copied: O_3530.jpg
Copied: O_342.jpg
Copied: O_1228.jpg
Copied: O_9456.jpg
Copied: O_6657.jpg
Copied: O_6184.jpg
Copied: O_386.jpg
Copied: O_3542.jpg
Copied: O_3263.jpg
Copied: O_5065.jpg
Copied: O_1156.jpg
Copied: O_1051.jpg
Copied: O_1145.jpg
Copied: O_6354.jpg
Copied: O_2545.jpg
Copied: O_6391.jpg
Copied: O_1243.jpg
Copied: O_6093.jpg
Copied: O_454.jpg
Copied: O_6760.jpg
Copied: O_2191.jpg
Copied: O_11499.jpg
Copied: O_7568.jpg
Copied: O_11859.jpg
Copied: O_5387.jpg
Copied: O_3208.jpg
Copied: O_10573.jpg
Copied: O_4153.jpg
Copied: O_7402.jpg
Copied: O_7726.jpg
Copied: O_2984.jpg
Copied: O_4082.jpg
Copied: O_11001.jpg
Copied: O_5301.jpg
Copied:

### Check for Duplicates

#### Helper: Auto Reconnect

In [ ]:
#@title Time Out Preventer (Advanced)
%%capture
AUTO_RECONNECT = True #@param {type:"boolean"}
#@markdown **Run this code to prevent Google Colab from Timeout**
from os import makedirs
makedirs("/root/.config/rclone", exist_ok = True)
if AUTO_RECONNECT:
  import IPython
  from google.colab import output

  display(IPython.display.Javascript('''
  function ClickConnect(){
    btn = document.querySelector("colab-connect-button")
    if (btn != null){
      console.log("Click colab-connect-button");
      btn.click()
      }

    btn = document.getElementById('ok')
    if (btn != null){
      console.log("Click reconnect");
      btn.click()
      }
    }

  setInterval(ClickConnect,60000)
  '''))

In [11]:
!pip install imagehash

In [12]:
from pathlib import Path
from PIL import Image
import imagehash
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from collections import defaultdict
import matplotlib.pyplot as plt

def get_resnet_embedding(img: Image.Image, model, transform, device):
    img_tensor = transform(img).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model(img_tensor).squeeze()
    return features.cpu()


def find_duplicates_images_with_end_deletion(
    dataset_dir: Path | str,
    hash_threshold: int = 5,
    sim_threshold: float = 0.98,
    delete: bool = False,
    show_preview: bool = True
):
    dataset_dir = Path(dataset_dir)
    print(f"🔍 Mendeteksi duplikat di dalam: {dataset_dir.resolve()}")

    # === Inisialisasi hash & CNN
    hash_dict = defaultdict(list)
    duplicates = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model = torch.nn.Sequential(*list(cnn_model.children())[:-1])
    cnn_model.eval().to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    counter = 0

    for img_path in dataset_dir.rglob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue
        try:
            img = Image.open(img_path).convert("RGB")
            h = imagehash.average_hash(img)

            # 🔁 Bandingkan dengan semua hash yang mirip
            for existing_hash in hash_dict:
                if h - existing_hash <= hash_threshold:
                    for existing_path in hash_dict[existing_hash]:
                        try:
                            img1 = Image.open(existing_path).convert("RGB")
                            emb1 = get_resnet_embedding(img1, cnn_model, transform, device)
                            emb2 = get_resnet_embedding(img, cnn_model, transform, device)

                            sim = F.cosine_similarity(emb1, emb2, dim=0).item()
                            if sim >= sim_threshold:
                                counter += 1
                                print(f"🔢 Kode duplikat: {counter}")
                                print(f"⚠️ Duplikat terdeteksi:")
                                print(f"  Original : {existing_path.name}")
                                print(f"  Duplicate: {img_path.name}")
                                print(f"  🔗 Cosine Similarity: {sim:.4f}")

                                duplicates.append(img_path)

                                if show_preview:
                                    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
                                    ax[0].imshow(img1)
                                    ax[0].set_title(f"Original: {existing_path.name}")
                                    ax[1].imshow(img)
                                    ax[1].set_title(f"Duplicate: {img_path.name}")
                                    for a in ax:
                                        a.axis("off")
                                    plt.tight_layout()
                                    plt.show()

                                print("\n")

                                break  # cukup validasi 1 yang cocok

                        except Exception as e:
                            print(f"❌ Error validasi CNN: {e}")
                    else:
                        continue
                    break  # keluar dari hash loop jika sudah match

            hash_dict[h].append(img_path)

        except Exception as e:
            print(f"❌ Gagal membuka {img_path}: {e}")

    # === Hapus jika diminta
    if delete:
        for dup in duplicates:
            try:
                dup.unlink()
                print(f"🗑️ Menghapus: {dup}")
            except Exception as e:
                print(f"❌ Gagal menghapus {dup}: {e}")

    print(f"\n✅ Total duplikat terverifikasi: {len(duplicates)}")
    return duplicates


def find_duplicates_images_with_immediate_deletion(
    dataset_dir: Path | str,
    hash_threshold: int = 5,
    sim_threshold: float = 0.98,
    delete: bool = False,
    show_preview: bool = True
):
    dataset_dir = Path(dataset_dir)
    print(f"🔍 Mendeteksi duplikat di dalam: {dataset_dir.resolve()}")

    hash_dict = defaultdict(list)
    duplicates = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    cnn_model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
    cnn_model = torch.nn.Sequential(*list(cnn_model.children())[:-1])
    cnn_model.eval().to(device)

    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor()
    ])

    counter = 0

    for img_path in dataset_dir.rglob("*.*"):
        if img_path.suffix.lower() not in [".jpg", ".jpeg", ".png"]:
            continue
        try:
            img = Image.open(img_path).convert("RGB")
            h = imagehash.average_hash(img)

            for existing_hash in hash_dict:
                if h - existing_hash <= hash_threshold:
                    for existing_path in hash_dict[existing_hash]:
                        try:
                            img1 = Image.open(existing_path).convert("RGB")
                            emb1 = get_resnet_embedding(img1, cnn_model, transform, device)
                            emb2 = get_resnet_embedding(img, cnn_model, transform, device)

                            sim = F.cosine_similarity(emb1, emb2, dim=0).item()
                            if sim >= sim_threshold:
                                counter += 1
                                print(f"🔢 Kode duplikat: {counter}")
                                print(f"⚠️ Duplikat terdeteksi:")
                                print(f"  Original : {existing_path.name}")
                                print(f"  Duplicate: {img_path.name}")
                                print(f"  🔗 Cosine Similarity: {sim:.4f}")

                                if show_preview:
                                    fig, ax = plt.subplots(1, 2, figsize=(8, 4))
                                    ax[0].imshow(img1)
                                    ax[0].set_title(f"Original: {existing_path.name}")
                                    ax[1].imshow(img)
                                    ax[1].set_title(f"Duplicate: {img_path.name}")
                                    for a in ax:
                                        a.axis("off")
                                    plt.tight_layout()
                                    plt.show()

                                if delete:
                                    try:
                                        img_path.unlink()
                                        print(f"🗑️ Duplikat dihapus: {img_path}")
                                    except Exception as e:
                                        print(f"❌ Gagal menghapus {img_path}: {e}")
                                else:
                                    duplicates.append(img_path)

                                print("\n")
                                break  # match pertama cukup
                        except Exception as e:
                            print(f"❌ Error validasi CNN: {e}")
                    else:
                        continue
                    break  # hash cocok, stop

            hash_dict[h].append(img_path)

        except Exception as e:
            print(f"❌ Gagal membuka {img_path}: {e}")

    print(f"\n✅ Total duplikat terverifikasi: {counter}")
    return duplicates

In [39]:
duplicates = find_duplicates_images_with_end_deletion(
    "dataset/train/organic",
    hash_threshold=6,
    delete=True,
    show_preview=False,
)

🔍 Mendeteksi duplikat di dalam: /kaggle/working/dataset/train/organic
🔢 Kode duplikat: 1
⚠️ Duplikat terdeteksi:
  Original : real_world_food_waste_0221__Image_127.png
  Duplicate: O_6023.jpg
  🔗 Cosine Similarity: 0.9902


🔢 Kode duplikat: 2
⚠️ Duplikat terdeteksi:
  Original : O_7726.jpg
  Duplicate: organic_011442_photo.jpg
  🔗 Cosine Similarity: 1.0000


🔢 Kode duplikat: 3
⚠️ Duplikat terdeteksi:
  Original : O_10673.jpg
  Duplicate: O_61.jpg
  🔗 Cosine Similarity: 0.9968


🔢 Kode duplikat: 4
⚠️ Duplikat terdeteksi:
  Original : O_275.jpg
  Duplicate: organic_005914_photo.jpg
  🔗 Cosine Similarity: 1.0000


🔢 Kode duplikat: 5
⚠️ Duplikat terdeteksi:
  Original : O_2854.jpg
  Duplicate: organic_006030_photo.jpg
  🔗 Cosine Similarity: 1.0000


🔢 Kode duplikat: 6
⚠️ Duplikat terdeteksi:
  Original : organic_010992_photo.jpg
  Duplicate: O_7320.jpg
  🔗 Cosine Similarity: 1.0000


🔢 Kode duplikat: 7
⚠️ Duplikat terdeteksi:
  Original : O_4890.jpg
  Duplicate: organic_004411_photo.jpg
  

### Check Dataset Distribution

In [13]:
!ls dataset/train

background  glass  metal  organic  paper  plastic  textiles


In [40]:
from collections import Counter
from torchvision.datasets import ImageFolder

train_dataset = ImageFolder("dataset/train")
label_counts = Counter([label for _, label in train_dataset])
print("Label mapping:", train_dataset.class_to_idx)
print("Distribusi kelas:", label_counts)

/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1043: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Label mapping: {'background': 0, 'glass': 1, 'metal': 2, 'organic': 3, 'paper': 4, 'plastic': 5, 'textiles': 6}
Distribusi kelas: Counter({2: 4420, 3: 4195, 5: 4051, 4: 4042, 1: 4035, 6: 3892, 0: 1507})


## Archive Dataset and Save to Google Drive

In [44]:
# !rm dataset.7z
# !7z a dataset_all-class-improved.7z dataset/

if USE_COLAB:
    !cp dataset.7z /content/drive/MyDrive/Cool Lee Yeah/8th Semester/Skripsi/Datasets/
else:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    GDRIVE_DATASET_FOLDER_ID = user_secrets.get_secret("GDRIVE_DATASET_FOLDER_ID")

    destination_folder_id = GDRIVE_DATASET_FOLDER_ID
    upload_target_file = "dataset_all-class-improved.7z"

    file_to_upload = drive.CreateFile(
        {
            "title": upload_target_file,
            "parents": [{"id": destination_folder_id}]
        }
    )

    print(f"⬆️ Uploading {file_to_upload['title']}...")
    file_to_upload.SetContentFile(upload_target_file)
    file_to_upload.Upload()
    print(f"✅ Uploaded successfully")


⬆️ Uploading dataset_all-class-improved.7z...
✅ Uploaded successfully
